# Worksheet 15 Solutions

## MCS 260 Fall 2020

## Problem 1

Write a program that uses the `threading` module to start a second thread.  The second thread should do the following:
* Print the numbers 30, 29, 28, ..., one per second, to the terminal.  Use `time.sleep(1)` to generate the delays.

Meanwhile, the main thread should do the following:
* Print a message "Type the entire alphabet and press enter"
* Wait for the user to enter a line of input
* Compare the input to "abcdefghijklmnopqrstuvwxyz".  If it is equal, print "Good job!".  If it is not equal, print "Better luck next time."

Test the program.  What happens if you take less than 30 seconds to enter a string?  Does the program exit immediately?

## Problem 1 solution

In [ ]:
"""
Wait for user to type alphabet, with a timer in a separate thread
"""
import threading
import time

# Worker thread function
def countdown():
    for i in range(30):
        print(30-i)
        time.sleep(1)
    print(0)

# Start the countdown thread
countdown_thread = threading.Thread(target=countdown)
countdown_thread.start()

g = input("Type the entire alphabet and press enter.\n")
# We ignore case, so "ABC..." is also ok.  That is optional.
if g.lower() == 'abcdefghijklmnopqrstuvwxyz':
    print("Good Job")
else:
    print("Better luck next time")

## Problem 2

In Python, each `threading.Thread` object has a method called `.is_alive()` which returns a boolean indicating whether the thread is still alive.  That is, it returns True if the thread is running, False if it has exited.

Modify the program from problem 1 as follows:
* Make the countdown thread a daemon thread (so it is terminated when the main thread is done)
* After reading the line of input, the main thread should check whether the countdown thread is still alive.  If it is, it should print the same message as before ("Good job!" or "Better luck next time.").  If it is not, meaning that at least 30 seconds elapsed, it should print "Time ran out."

## Problem 2 solution

In [ ]:
"""
Wait for user to type alphabet, with a timer in a separate thread
"""
import threading
import time

# Worker thread function
def countdown():
    for i in range(10):
        print(10-i)
        time.sleep(1)

# Start the countdown thread
# We could add daemon=True in the Thread constructor to make
# the program exit when the main thread is done.  As is, it
# will not exit until the countdown is complete.
countdown_thread = threading.Thread(target=countdown,daemon=True)
countdown_thread.start()

g = input("Type the entire alphabet and press enter.\n")

if not countdown_thread.is_alive():
    print("Time ran out.")
elif g.lower() == 'abcdefghijklmnopqrstuvwxyz':
    print("Good Job")
else:
    print("Better luck next time")

## Problem 3

Write a tkinter GUI program called `squares.py`.  The interface should have a slider and a read-only entry widget.  The slider should allow the user to select an integer between 0 and 30.  The entry widget should display a statement about the square of the current value of the slider, like `7**2 = 49` if the current slider value is `5`.  The final GUI might look like this:

![Screenshot of squares GUI](squares-screenshot.png)

(The exact appearance will vary depending on the platform you use, i.e. Windows, MacOS, Linux, ...)

## Problem 3 Solution

In [ ]:
'''
tkinter GUI for the function f(x) = x**2
'''

import tkinter
import tkinter.ttk

class SquareApp(tkinter.Tk):
    """
    Squaring GUI main window
    """
    def __init__(self,name="Squares"):
        super().__init__()
        self.title(name)

        self.n = tkinter.IntVar()
        # If we make self.n the variable for an Entry widget, it will
        # display floats instead of ints, reflecting the fractional
        # position of the slider.  To get ints, we write a callback
        # to update the string in the Entry box.
        self.n.trace_add("write",lambda a,b,c:self.nchg())

        # Inserts the scale into the GUI, with range 0 to 30
        self.scale = tkinter.ttk.Scale(self,variable=self.n,from_=0,to=30,length=200) #,command=lambda x:self.update_scale_display())
        self.scale.grid(row=0,column=0,sticky="ew",padx=10,pady=10)

        # Displays the result in an entry box
        self.s = tkinter.StringVar()
        self.scale_show =  tkinter.ttk.Entry(self,state="readonly",textvariable=self.s,width=12)
        self.scale_show.grid(row=0,column=1,sticky="w",padx=10,pady=10)

        self.nchg()
    
    def nchg(self):
        """Callback when input value n has changed, to update the output string"""
        n=self.n.get()
        self.s.set("{}**2 = {}".format(n,n**2))

# Create and start the main window
app = SquareApp()
app.mainloop()


## Problem 4

Modify the producer-consumer example program from lecture 39,

* [workers_10am_fixed.py](https://github.com/emilydumas/mcs260fall2020/blob/master/samplecode/threads/workers_10am_fixed.py)
* [workers_2pm_fixed.py](https://github.com/emilydumas/mcs260fall2020/blob/master/samplecode/threads/workers_2pm.py)

in the following ways:

* Have two queues: `job_queue` to send work to the workers, and `results_queue` that the workers use to send results back to the main thread.
* Change the workers so they don't just print the job number.  Instead, they compute the square of the job number, wait a short time, and then send the square back to the main thread using `results_queue`.
* After starting the workers, the main queue adds lots of jobs all at once.  Then it waits for results to be returned, displaying them as they arrive.  (It might print `Received result: 49` or something similar.)  Use `queue.Queue.get()` for this; this function will not return until a new item is available.

For simplicity, in this problem you can write a program where the main thread will run forever, waiting for more results even after the last one has been received.  This allows you to not worry about monitoring the state of the worker threads so that the main thread can decide when to exit.

## Problem 4 solution

In [ ]:
"""Threading example with two queues to communicate with worker threads"""
import threading
import time
import queue

def worker_main(name,Q,R):
    """
    Main function of the worker threads
    `name` is the name of this thread
    `Q` is the queue from which it takes jobs
    `R` is the queue to which it submits results
    """
    print("Worker {} starting".format(name))
    while True:
        # Retrieve work from the queue (or block until available)
        job = Q.get()
        # Simulate this job taking some time
        time.sleep(1)
        result = job**2; # square the given number
        print("Worker {} sending square of job {} back to main thread\n".format(name,job),flush=True)
        R.put(result) # send the result back to the main thread

job_queue = queue.Queue()
results_queue = queue.Queue()

workers = [ 
    threading.Thread(target=worker_main,args=(i,job_queue,results_queue)) for i in range(2)
]
for w in workers:
    w.start()

for i in range(10):
    print("Main submitting job {} to the queue".format(i))
    job_queue.put(i)

print("Main waiting for results...")
while True:
    result = results_queue.get()
    print("Main received result {}".format(result),flush = True)

## Problem 5

Modify the program from Problem 3 so that there are two sliders, and so that the sliders are labeled.  One slider should be called `Base`, the other `Exponent`.  The Entry widget should show the computation of `b**k` where `b` is the base and `k` is the exponent.  For example, if the base slider is set to 5 and the exponent slider to 3, it would show `5**3 = 125`.

Also add a button to quit the program that is shown below the rest of the interface, on the right hand side.

## Problem 5 solution

In [ ]:
'''
tkinter GUI for the function f(x,k) = x**k
'''

import tkinter
import tkinter.ttk

class PowerApp(tkinter.Tk):
    """
    Power GUI main window
    """
    def __init__(self,name="Powers"):
        super().__init__()
        self.title(name)

        self.n = tkinter.IntVar()
        self.k = tkinter.IntVar()
        self.n.trace_add("write",lambda a,b,c:self.recalculate())
        self.k.trace_add("write",lambda a,b,c:self.recalculate())

        self.base_label = tkinter.ttk.Label(self,text="Base")
        self.base_label.grid(row=0,column=0)

        self.exponent_label = tkinter.ttk.Label(self,text="Exponent")
        self.exponent_label.grid(row=1,column=0)

        # Slider for the base
        self.base_scale = tkinter.ttk.Scale(self,variable=self.n,from_=0,to=30,length=200) #,command=lambda x:self.update_scale_display())
        self.base_scale.grid(row=0,column=1,sticky="ew",padx=10,pady=10)

        # Slider for the exponent
        self.exp_scale = tkinter.ttk.Scale(self,variable=self.k,from_=1,to=5,length=200) #,command=lambda x:self.update_scale_display())
        self.exp_scale.grid(row=1,column=1,sticky="ew",padx=10,pady=10)

        # Displays the result in an entry box
        self.s = tkinter.StringVar()
        self.scale_show = tkinter.ttk.Entry(self,state="readonly",textvariable=self.s,width=18)
        self.scale_show.grid(row=0,column=2,sticky="ew",padx=10,pady=10)

        # Exit button
        self.exit_button = tkinter.ttk.Button(self,text="Exit",command=exit)
        self.exit_button.grid(row=2,column=2,padx=10,pady=10,sticky="e")

        self.recalculate()

    def recalculate(self):
        """Callback when one of the sliders changes"""
        n=self.n.get()
        k=self.k.get()
        self.s.set("{}**{} = {}".format(n,k,n**k))

app = PowerApp()
app.mainloop()